In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("MovieGenre.csv")
print(df.shape)
df.head(5)


(40108, 6)


,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [13]:
#ensure imdbid is unique for all
print(df['imdbId'].is_unique)
print(len(df['imdbId']) - df['imdbId'].nunique())

False
593


In [14]:
#remove not unique values
df = df.drop_duplicates(subset=['imdbId'], keep=False)
print(df['imdbId'].is_unique)
print(len(df['imdbId']) - df['imdbId'].nunique())

True
0
